<a href="https://colab.research.google.com/github/mohamedahmedsaadahmed77/Research-Project-Selected-2/blob/master/LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [0]:
!git clone https://github.com/mohamedahmedsaadahmed77/OCR-Selected-2.git

In [0]:
import numpy as np
import pandas as pd
import os
from imutils import paths
import cv2

def load_images(path):
  print("[INFO] loading images...")
  imagePaths = list(paths.list_images(path))
  imagePaths.sort()
  data = []
  labels = []
  for imagePath in imagePaths:
    label = int(imagePath.split(os.path.sep)[-2])
    image = cv2.imread(imagePath, 0)
    image = np.reshape(image, (28*28))/255.0
    data.append(image)
    labels.append(label)
  data = np.array(data)
  labels=pd.get_dummies(labels)
  labels = np.array(labels)
  print("[INFO] done")
  return data,labels

In [0]:
x_train, y_train = load_images('/content/OCR-Selected-2/Dataset/Training')
x_test, y_test = load_images('/content/OCR-Selected-2/Dataset/Test')

In [0]:
# Hyper parameters
learning_rate = 0.0001
epochs = 32
batch_size = 32
batches = int(x_train.shape[0] / batch_size)


# I choose placeholder to make it recieve any number of records to make code flexable
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# I choose variable cause it’s values will be changed to get more better values and i put random values for the first time
W = tf.Variable(.1 * np.random.randn(784, 10).astype(np.float32))
B = tf.Variable(.1 * np.random.randn(10).astype(np.float32))

In [0]:
# Formula of logistic regression : X * W + B
pred = tf.nn.softmax(tf.add(tf.matmul(X,W), B))

# Cost function
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(pred), axis=1))

# Optimiser that will get better values for bais and weights
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [0]:
# Session to determine the flow of computitional graph (TensorFlow)
with tf.Session() as sesh:
    # Set initial values of tensor variables (Mandatory)
    sesh.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        for i in range(batches):
            offset = i * epoch
            x = x_train[offset: offset + batch_size]
            y = y_train[offset: offset + batch_size]
            sesh.run(optimizer, feed_dict={X: x, Y:y})
            # Get cost function value (Optional)
            costVal = sesh.run(cost, feed_dict={X:x, Y:y})
        print(f'epoch: {epoch:2d} cost_val= {costVal:.4f}')
    
    # Calculate the accuracy
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    acc = accuracy.eval({X: x_test, Y: y_test})
    print(f'Accuracy: {acc * 100:.2f}%')